<a href="https://colab.research.google.com/github/Alymostafa/Nlp_task/blob/main/Best_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##weights and biases

In [1]:
!pip install  wandb

     |████████████████████████████████| 1.7MB 4.5MB/s 
     |████████████████████████████████| 163kB 29.5MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 122kB 28.4MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=f2b4c86f2a587ed3e2de5ec2d645cbd7e23c84b54dc847cf9693ccb6bd38130c
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=de7d56a35930f8ea2c5e9f30a97aaa694b6330a23867aad9ee2ccc32debd28dd
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=134a6ea9299bb16a90d2c81840ae75b521df1f0b65af

In [3]:
!wandb login 

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##imports

In [42]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn import metrics
from pprint import pprint
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
import nltk
nltk.download('names')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re 
from nltk.corpus import stopwords
#from sklearn.datasets import fetch_rcv1


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##<h1>First DataSets</h1>

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
def twenty_newsgroup_to_csv(data,name):
    newsgroups_train = fetch_20newsgroups(subset= data , remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names)
    targets.columns=['title']

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out['date'] = pd.to_datetime('now')
    out.to_csv(name+'.csv')

twenty_newsgroup_to_csv('train','train')
twenty_newsgroup_to_csv('test','test')

In [ ]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df.dropna(subset = ["text"], inplace=True)
df_test.dropna(subset = ["text"], inplace=True)

In [ ]:
all_names = names.words()
WNL = WordNetLemmatizer()
def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1 
    return(list(cleaned.values()))

In [ ]:
df['text'] = clean(df['text'])
df_test['text'] = clean(df_test['text'])

In [ ]:
print(len(df))
print(len(df_test))


11096
7370


In [ ]:
df.drop(labels=['target','date'],axis=1,inplace=True)


In [ ]:
df_test.drop(labels=['target','date'],axis=1,inplace=True)


##<h1>Second Dataset</h1>

In [ ]:
nltk.download('reuters')
nltk.download('punkt')
from nltk.corpus import stopwords, reuters

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
documents = reuters.fileids()
 
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
                            documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"),
                           documents))
 
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]

In [ ]:
docs_pdf_train = pd.DataFrame(train_docs) 
docs_pdf_test = pd.DataFrame(test_docs) 
docs_pdf_train['text'] = pd.DataFrame(train_docs) 
docs_pdf_test['text'] = pd.DataFrame(test_docs) 
docs_pdf_train.drop([docs_pdf_train.columns[0]], axis ='columns',inplace=True)
docs_pdf_test.drop([docs_pdf_test.columns[0]], axis ='columns',inplace=True)
docs_pdf_train.dropna(inplace=True)
docs_pdf_test.dropna(inplace=True)

In [ ]:
docs_pdf_train.dropna(subset = ["text"], inplace=True)
docs_pdf_train.replace(r'-', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'\t', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'\n', ' ', regex=True,inplace=True)
docs_pdf_train.replace(r'   ', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'-', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'\t', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'\n', ' ', regex=True,inplace=True)
docs_pdf_test.replace(r'   ', ' ', regex=True,inplace=True)

In [ ]:
docs_pdf_train['text'] = clean(docs_pdf_train['text'])
docs_pdf_test['text'] = clean(docs_pdf_test['text'])

In [ ]:
train_labels = [reuters.categories(doc_id)for doc_id in train_docs_id]
test_labels = [reuters.categories(doc_id)for doc_id in test_docs_id]

In [ ]:
achedStopWords = stopwords.words("english")
 
def tokenize(text):
  min_length = 3
  words = map(lambda word: word.lower(), word_tokenize(text))
  words = [word for word in words if word not in achedStopWords]
  tokens = (list(map(lambda token: PorterStemmer().stem(token),
                                   words)))
  p = re.compile('[a-zA-Z]+');
  filtered_tokens =list(filter (lambda token: p.match(token) and
                               len(token) >= min_length,
                               tokens))
  return filtered_tokens

def word_tokenizer():
  tokenize_docs = []
  docs_tokens = docs_pdf_train['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
tokenize_docs = word_tokenizer()

In [ ]:
def word_tokenizer_test():
  tokenize_docs = []
  docs_tokens = docs_pdf_test['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
tokenize_docs_test = word_tokenizer_test()

In [ ]:
df_train_2=pd.DataFrame(docs_pdf_train,columns=['text'])
df_train_2['target']=''
df_train_2['target']=train_labels


In [ ]:
df_test_2=pd.DataFrame(docs_pdf_test,columns=['text'])
df_test_2['target']=''
df_test_2['target']=test_labels
df_test_2

,text,target
0,asian exporter fear damage from japan rift mou...,[trade]
1,china daily say vermin eat pct grain stock a s...,[grain]
2,japan to revise long term energy demand downwa...,"[crude, nat-gas]"
3,thai trade deficit widens in first quarter tra...,"[corn, grain, rice, rubber, sugar, tin, trade]"
4,indonesia see cpo price rising sharply indones...,"[palm-oil, veg-oil]"
...,...,...
3014,chase corp make offer for entregrowth corp ltd...,[acq]
3015,tokyo dealer see dollar poised to breach yen f...,"[dlr, money-fx, yen]"
3016,conference cut gulf war risk charge the pakist...,[ship]
3017,soviet industrial slower in the soviet industr...,[ipi]


In [ ]:
df_train_2.rename(columns={'target':'title'},inplace=True)
df_test_2.rename(columns={'target':'title'},inplace=True)


In [ ]:
frames_train=[df,df_train_2]
frames_test=[df_test,df_test_2]
all_dataframes_train=pd.concat(frames_train,keys=['x','y'])
all_dataframes_test=pd.concat(frames_test,keys=['x','y'])


In [ ]:
all_dataframes_train.drop(labels='Unnamed: 0',inplace=True,axis=1)
all_dataframes_test.drop(labels='Unnamed: 0',inplace=True,axis=1)

##Bert

In [1]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="nlp-task")

wandb: Currently logged in as: alymostafa (use `wandb login --relogin` to force relogin)


In [2]:
!pip install 'kashgari>=2.0.0'	

     |████████████████████████████████| 92kB 3.9MB/s 
     |████████████████████████████████| 24.2MB 130kB/s 
     |████████████████████████████████| 40kB 5.5MB/s 
     |████████████████████████████████| 378kB 57.8MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
  Created wheel for bert4keras: filename=bert4keras-0.8.8-cp36-none-any.whl size=39383 sha256=47a38b8758bbacfc41d50b17f71aeba3de01727a8a8fd83c1a2b34ded5cacb7b
  Stored in directory: /root/.cache/pip/wheels/6f/07/52/94c66529c3cee3757d96608bb593adbe623b6f29881bec342b
Successfully built bert4keras
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [3]:
import logging
from kashgari.embeddings import BertEmbedding
from kashgari.tasks.classification import BiGRU_Model
from kashgari.tasks.classification import BiLSTM_Model

In [4]:
logging.basicConfig(level='DEBUG')

In [5]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-10-12 06:32:28--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   137MB/s    in 2.8s    

2020-10-12 06:32:31 (137 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [6]:
!unzip '/content/uncased_L-12_H-768_A-12.zip'

Archive:  /content/uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [7]:
bert_embed = BertEmbedding('/content/uncased_L-12_H-768_A-12')

2020-10-12 06:32:36,162 [DEBUG] kashgari - ------------------------------------------------
2020-10-12 06:32:36,163 [DEBUG] kashgari - Loaded transformer model's vocab
2020-10-12 06:32:36,163 [DEBUG] kashgari - config_path       : /content/uncased_L-12_H-768_A-12/bert_config.json
2020-10-12 06:32:36,164 [DEBUG] kashgari - vocab_path      : /content/uncased_L-12_H-768_A-12/vocab.txt
2020-10-12 06:32:36,164 [DEBUG] kashgari - checkpoint_path : /content/uncased_L-12_H-768_A-12/bert_model.ckpt
2020-10-12 06:32:36,165 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31

In [8]:
model = BiLSTM_Model(bert_embed)

In [ ]:
toknize_train=all_dataframes_train['text'].tolist()
toknize_train=[i.split(' ') for i in toknize_train]

NameError: ignored

In [ ]:
modifed_list_labels=all_dataframes_train['title'][:11096].tolist()
modifed_list_labels=[[i] for i in modifed_list_labels]


NameError: ignored

In [ ]:
all_dataframes_train['title'][:11096]=modifed_list_labels
train_labels_=all_dataframes_train['title'].tolist()


In [ ]:
toknize_test=all_dataframes_test['text'].tolist()
toknize_test=[i.split(' ') for i in toknize_test]


In [ ]:
modifed_list_labels_test=all_dataframes_test['title'][:7370].tolist()
modifed_list_labels_test=[[i] for i in modifed_list_labels_test]
all_dataframes_test['title'][:7370]=modifed_list_labels_test
test_labels_=all_dataframes_test['title'].tolist()


##Wikipedia Dataset

In [9]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
!wget 'https://www.dropbox.com/s/dkfqa5otfngdl0o/all_dataset_fin%20%281%29.csv?dl=0' -O all_dataset_fin.csv

--2020-10-12 06:36:01--  https://www.dropbox.com/s/dkfqa5otfngdl0o/all_dataset_fin%20%281%29.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/dkfqa5otfngdl0o/all_dataset_fin%20%281%29.csv [following]
--2020-10-12 06:36:01--  https://www.dropbox.com/s/raw/dkfqa5otfngdl0o/all_dataset_fin%20%281%29.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfdc5165b4661b0a64c7c98628f.dl.dropboxusercontent.com/cd/0/inline/BBFZF5bzt7qvLKba3BEb-oSQGb3BJQ9X6qo39yC8AI3qT5_BIbSJtGC_1KklhOl-tu7UWdtMpvkOwSPgd852vXUIPNu34XhrVR701_y0bhwQXbY_sweZEPBdJgmNhF1ryVc/file# [following]
--2020-10-12 06:36:02--  https://ucfdc5165b4661b0a64c7c98628f.dl.dropboxusercontent.com/cd/0/inline/BBFZF5bzt7qvLKba3BEb-oSQGb3BJQ9X6qo39yC8AI3qT5_BIbSJtGC_1K

In [32]:
!wget 'https://storage.googleapis.com/wandb-production.appspot.com/alymostafa/nlp-task/1mk6uv86/files/lstm/embed_model_weights.h5?Expires=1602484706&GoogleAccessId=gorilla-cloud-storage%40wandb-production.iam.gserviceaccount.com&Signature=gFAQhgcGf5ZsSAO3wNA1pkhaFRJ%2B4GrVSDISga8D1DBMBDAR%2B2Bn8A2P1qO9GhigGDwhTYNm1yV%2F9OoNAlwnbZPKvIMfKKE%2Bn0ncw7E2LJvRAuUTFMJxyPTQS8ocPLBrbx8kD7L98opmHZ%2Ft2LQLToQFfC5BgPZuzKgQ8W9H1Y4CjgGlL%2F2%2FYaJHmipcD6dIjsKx6Ig9yNjaC47bA6xZM8UE%2B%2BqmOV4US2ebpD3230zKTBQC9N%2B5nAetismkOFaMhSkbXwUKfPJTqZQvx1Nwauc4XOV0J71MsAl7e%2F%2BXDWQrDzcntN%2F6XZrauZZpzlldMtH1z92ArSlCpg3XWjdxkg%3D%3D' -O embed_model_weights.h5

--2020-10-12 06:38:05--  https://storage.googleapis.com/wandb-production.appspot.com/alymostafa/nlp-task/1mk6uv86/files/lstm/embed_model_weights.h5?Expires=1602484706&GoogleAccessId=gorilla-cloud-storage%40wandb-production.iam.gserviceaccount.com&Signature=gFAQhgcGf5ZsSAO3wNA1pkhaFRJ%2B4GrVSDISga8D1DBMBDAR%2B2Bn8A2P1qO9GhigGDwhTYNm1yV%2F9OoNAlwnbZPKvIMfKKE%2Bn0ncw7E2LJvRAuUTFMJxyPTQS8ocPLBrbx8kD7L98opmHZ%2Ft2LQLToQFfC5BgPZuzKgQ8W9H1Y4CjgGlL%2F2%2FYaJHmipcD6dIjsKx6Ig9yNjaC47bA6xZM8UE%2B%2BqmOV4US2ebpD3230zKTBQC9N%2B5nAetismkOFaMhSkbXwUKfPJTqZQvx1Nwauc4XOV0J71MsAl7e%2F%2BXDWQrDzcntN%2F6XZrauZZpzlldMtH1z92ArSlCpg3XWjdxkg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 435886848 (416M) [application/keras]
Saving to: ‘embed_model_weights.h5’

embed_model_weights 100%[=======

In [33]:
!wget 'https://storage.googleapis.com/wandb-production.appspot.com/alymostafa/nlp-task/1mk6uv86/files/lstm/model_config.json?Expires=1602484769&GoogleAccessId=gorilla-cloud-storage%40wandb-production.iam.gserviceaccount.com&Signature=qbLjOliKF%2BLx0T%2BRUurlVWwiSdIFmsdYFSJL0YVnCpI9dhT%2Bl1G2mYZcT9RIZJ%2BRoqXruosMCR5IjRd4%2BErMg0C7f%2BT3mTh%2Bggh2SH8%2FA5ZyK%2FJh%2BWRqJPa9pZSc6SF%2BAonsqBeQHsgqG277%2B3GwRyu4cWs%2BkAVLEw9QWWfLcw%2F%2FdRsbHXeQxHZurL%2F39%2BPNYBRdjx6p2SYZk9JhENJn8dobaQ%2FS3UIkpsK%2FNxeoZkd3d6wG%2BzcU7hLzwFI9Ua0tIBI6gakJqwD1HMf3O3sRUL6HaynsYHfd%2FhSW%2FNZJREf77cvSUyJA71zhB%2FD0MTq%2BIRfz3zJ%2F7g5OQxFv0swGQw%3D%3D' -O model_config.json

--2020-10-12 06:39:12--  https://storage.googleapis.com/wandb-production.appspot.com/alymostafa/nlp-task/1mk6uv86/files/lstm/model_config.json?Expires=1602484769&GoogleAccessId=gorilla-cloud-storage%40wandb-production.iam.gserviceaccount.com&Signature=qbLjOliKF%2BLx0T%2BRUurlVWwiSdIFmsdYFSJL0YVnCpI9dhT%2Bl1G2mYZcT9RIZJ%2BRoqXruosMCR5IjRd4%2BErMg0C7f%2BT3mTh%2Bggh2SH8%2FA5ZyK%2FJh%2BWRqJPa9pZSc6SF%2BAonsqBeQHsgqG277%2B3GwRyu4cWs%2BkAVLEw9QWWfLcw%2F%2FdRsbHXeQxHZurL%2F39%2BPNYBRdjx6p2SYZk9JhENJn8dobaQ%2FS3UIkpsK%2FNxeoZkd3d6wG%2BzcU7hLzwFI9Ua0tIBI6gakJqwD1HMf3O3sRUL6HaynsYHfd%2FhSW%2FNZJREf77cvSUyJA71zhB%2FD0MTq%2BIRfz3zJ%2F7g5OQxFv0swGQw%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 74.125.199.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 945225 (923K) [application/json]
Saving to: ‘model_config.json’

model_config.json  

In [34]:
!wget 'https://storage.googleapis.com/wandb-production.appspot.com/alymostafa/nlp-task/1mk6uv86/files/lstm/model_weights.h5?Expires=1602484824&GoogleAccessId=gorilla-cloud-storage%40wandb-production.iam.gserviceaccount.com&Signature=B25KSYQ84dQexkR681l16pGwNzh6jSEoUE3LDd7gIiuxQW00elfnbW%2Fb1Tk3W8DhO9KNqLfK7dmvbtdyQ9HPuUF9jOfdY3PxTOdDJxmXVatZey74U5URhog%2BRCrYGeRexrqYkmj2MmoyMo2YCu9EASORkTQTg0gPi%2BOe0W%2FA5TZe3TXgmENhvFGsQDeuAeT50ygiHQmPMc3c58w%2FC%2FtoyHqvMfAdPgLvUbYfcWym940W1VLiV%2BC8GjqYbV04Tdgem86fHY4AfOQsIJ7YyAa6kZ%2Bat1V6%2FNj670Ge2Tp5mG6HhmcHsskzMX9S9y%2BgId9BKwk%2Fbgg982RdF5i6xwFlrQ%3D%3D' -O model_weights.h5

--2020-10-12 06:39:49--  https://storage.googleapis.com/wandb-production.appspot.com/alymostafa/nlp-task/1mk6uv86/files/lstm/model_weights.h5?Expires=1602484824&GoogleAccessId=gorilla-cloud-storage%40wandb-production.iam.gserviceaccount.com&Signature=B25KSYQ84dQexkR681l16pGwNzh6jSEoUE3LDd7gIiuxQW00elfnbW%2Fb1Tk3W8DhO9KNqLfK7dmvbtdyQ9HPuUF9jOfdY3PxTOdDJxmXVatZey74U5URhog%2BRCrYGeRexrqYkmj2MmoyMo2YCu9EASORkTQTg0gPi%2BOe0W%2FA5TZe3TXgmENhvFGsQDeuAeT50ygiHQmPMc3c58w%2FC%2FtoyHqvMfAdPgLvUbYfcWym940W1VLiV%2BC8GjqYbV04Tdgem86fHY4AfOQsIJ7YyAa6kZ%2Bat1V6%2FNj670Ge2Tp5mG6HhmcHsskzMX9S9y%2BgId9BKwk%2Fbgg982RdF5i6xwFlrQ%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439580724 (419M) [application/keras]
Saving to: ‘model_weights.h5’

model_weights.h5    100%[===================>] 4

In [14]:
df=pd.read_csv("/content/all_dataset_fin.csv")

In [15]:
df

,Unnamed: 0,title,text
0,0,Academic_Science,"['an', 'academic', 'degree', 'is', 'a', 'quali..."
1,1,Academic_Science,"['an', 'alumnus', 'masculine', 'plural', 'alum..."
2,2,Academic_Science,"['an', 'example', 'of', 'an', 'amateur', 'radi..."
3,3,Academic_Science,"['anthropology', 'is', 'the', 'study', 'of', '..."
4,4,Academic_Science,"['roman', 'ruins', 'lausanne', 'switzerland', ..."
...,...,...,...
158,158,Medical,"['surgeons', 'repairing', 'a', 'ruptured', 'ac..."
159,159,Medical,"['a', 'syndrome', 'is', 'a', 'set', 'of', 'med..."
160,160,Medical,"['therapy', 'often', 'abbreviated', 'tx', 'tx'..."
161,161,Medical,"['organ', 'transplantation', 'is', 'a', 'medic..."


In [16]:
df[df['title']=='Miscellaneous']

,Unnamed: 0,title,text
116,116,Miscellaneous,"['the', 'miscellaneous', 'was', 'a', 's', 'alt..."
117,117,Miscellaneous,"['nasa', 'test', 'aircraft', 'the', 'mil', 'mi..."
118,118,Miscellaneous,"['aviation', 'is', 'the', 'practical', 'aspect..."
119,119,Miscellaneous,"['animals', 'are', 'multicellular', 'eukaryoti..."
120,120,Miscellaneous,"['some', 'of', 'the', 'trophies', 'earned', 'b..."
121,121,Miscellaneous,"['obverse', 'of', 'medal', 'distributed', 'by'..."
122,122,Miscellaneous,"['farmland', 'in', 'the', 'united', 'states', ..."
123,123,Miscellaneous,"['fields', 'in', 'z', 'horie', 'slovakia', 'a'..."
124,124,Miscellaneous,"['various', 'foods', 'foods', 'from', 'plant',..."
125,125,Miscellaneous,"['the', 'nutrition', 'facts', 'table', 'displa..."


In [17]:
df['title'].unique()

array(['Academic_Science', 'Business', 'Community', 'Computing',
       'Governmental', 'Miscellaneous', 'Medical'], dtype=object)

In [18]:
Academic_Science = df[df['title']=='Academic_Science']
Business = df[df['title']=='Business']
Community = df[df['title']=='Community']
Computing = df[df['title']=='Computing']
Governmental = df[df['title']=='Governmental']
Miscellaneous = df[df['title']=='Miscellaneous']
Medical = df[df['title']=='Medical']

In [19]:
list_Academic_Science = list(Academic_Science['text'])
list_Business = list(Business['text'])
list_Community = list(Community['text'])
list_Computing = list(Computing['text'])
list_Governmental = list(Governmental['text'])
list_Miscellaneous = list(Miscellaneous['text'])
list_Medical = list(Medical['text'])

In [20]:
import ast
list_Academic_Science=[ast.literal_eval(i) for i in list_Academic_Science]
list_Business=[ast.literal_eval(i) for i in list_Business]
list_Community=[ast.literal_eval(i) for i in list_Community]
list_Computing=[ast.literal_eval(i) for i in list_Computing]
list_Governmental=[ast.literal_eval(i) for i in list_Governmental]
list_Miscellaneous=[ast.literal_eval(i) for i in list_Miscellaneous]
list_Medical=[ast.literal_eval(i) for i in list_Medical]

In [21]:
seq = 150
count = 0
list_list_Aca=[]
list_Aca= [] 
for w in list_Academic_Science:
  for j in w:
    list_Aca.append(j)
    count = count + 1
    if count == seq:
      list_list_Aca.append(list_Aca)
      count = 0
      list_Aca = []


count = 0
list_list_bus=[]
list_bus= [] 
for w in list_Business:
  for j in w:
    list_bus.append(j)
    count = count + 1
    if count == seq:
      list_list_bus.append(list_bus)
      count = 0
      list_bus = []


count = 0
list_list_comm=[]
list_comm= [] 
for w in list_Community:
  for j in w:
    list_comm.append(j)
    count = count + 1
    if count == seq:
      list_list_comm.append(list_comm)
      count = 0
      list_comm = []


count = 0
list_list_comp=[]
list_comp= [] 
for w in list_Computing:
  for j in w:
    list_comp.append(j)
    count = count + 1
    if count == seq:
      list_list_comp.append(list_comp)
      count = 0
      list_comp = []

count = 0
list_list_cov=[]
list_cov= [] 
for w in list_Governmental:
  for j in w:
    list_cov.append(j)
    count = count + 1
    if count == seq:
      list_list_cov.append(list_cov)
      count = 0
      list_cov = []


count = 0
list_list_mis=[]
list_mis= [] 
for w in list_Miscellaneous:
  for j in w:
    list_mis.append(j)
    count = count + 1
    if count == seq:
      list_list_mis.append(list_mis)
      count = 0
      list_mis = []


count = 0
list_list_med=[]
list_med= [] 
for w in list_Medical:
  for j in w:
    list_med.append(j)
    count = count + 1
    if count == seq:
      list_list_med.append(list_med)
      count = 0
      list_med = []




In [22]:
df_aca = pd.DataFrame()
df_aca['text'] = list_list_Aca
df_aca['title'] = "acadamic_science"

df_bus = pd.DataFrame()
df_bus['text'] = list_list_bus
df_bus['title'] = "Business"

df_comm = pd.DataFrame()
df_comm['text'] = list_list_comm
df_comm['title'] = "community"

df_comp = pd.DataFrame()
df_comp['text'] = list_list_comp
df_comp['title'] = "computing"

df_gov = pd.DataFrame()
df_gov['text'] = list_list_cov
df_gov['title'] = "govermnetal"

df_mis = pd.DataFrame()
df_mis['text'] = list_Miscellaneous
df_mis['title'] = "Miscellaneous"

df_med = pd.DataFrame()
df_med['text'] = list_list_med
df_med['title'] = "medical"

In [23]:
df_all_cat = pd.concat([df_aca,df_bus,df_comm,df_comp,df_gov,df_mis,df_med],ignore_index = True)

In [24]:
df_all_cat['text'] = df_all_cat['text'].apply(lambda x: [item for item in x if item not in stop_words])

In [25]:
df_all_cat

,text,title
0,"[academic, degree, qualification, awarded, stu...",acadamic_science
1,"[master, degree, doctorate, adopted, diverse, ...",acadamic_science
2,"[ecclesiastic, scholastic, right, remained, bo...",acadamic_science
3,"[master, doctor, latin, meaning, literally, te...",acadamic_science
4,"[due, renaissance, conviction, real, knowledge...",acadamic_science
...,...,...
4699,"[society, meeting, resolved, promote, study, f...",medical
4700,"[early, th, century, boston, new, york, philad...",medical
4701,"[designated, exceptions, paraveterinary, worke...",medical
4702,"[way, prior, kittens, adoption, paraveterinary...",medical


In [26]:
df_all_cat[df_all_cat['title']=='acadamic_science']

,text,title
0,"[academic, degree, qualification, awarded, stu...",acadamic_science
1,"[master, degree, doctorate, adopted, diverse, ...",acadamic_science
2,"[ecclesiastic, scholastic, right, remained, bo...",acadamic_science
3,"[master, doctor, latin, meaning, literally, te...",acadamic_science
4,"[due, renaissance, conviction, real, knowledge...",acadamic_science
...,...,...
1533,"[biology, entered, evolutionary, biology, init...",acadamic_science
1534,"[system, outdated, modern, alternative, classi...",acadamic_science
1535,"[classification, system, called, linnaean, tax...",acadamic_science
1536,"[earth, focusing, topics, like, plate, tectoni...",acadamic_science


In [27]:
msk = np.random.rand(len(df_all_cat)) < 0.8
train_df = df_all_cat[msk]
test_val_df = df_all_cat[~msk]

In [28]:
msk = np.random.rand(len(test_val_df)) < 0.5
test_df = test_val_df[msk]
validate_df = test_val_df[~msk]

In [29]:
validate_df[validate_df['title']=='Miscellaneous']

,text,title
3841,"[foundation, may, refer, literature, foundatio...",Miscellaneous
3846,"[shipping, line, business, transports, cargo, ...",Miscellaneous
3847,"[sailing, employs, wind, acting, sails, wingsa...",Miscellaneous


In [30]:
print(f'train data count: {len(train_df)}')
print(f'test data count: {len(test_df)}')
print(f'validate data count: {len(validate_df)}')
train_x, train_y = list(train_df['text']), list(train_df['title'])
test_x, test_y = list(test_df['text']), list(test_df['title'])
validate_x, validate_y = list(validate_df['text']), list(validate_df['title'])

train data count: 3739
test data count: 475
validate data count: 490


In [24]:
model.fit(train_x,train_y,validate_x,validate_y,epochs=20, callbacks=[WandbCallback()])

Preparing text vocab dict: 100%|██████████| 331/331 [00:00<00:00, 22953.80it/s]
2020-10-12 04:21:14,307 [DEBUG] kashgari - --- Build vocab dict finished, Total: 14327 ---
2020-10-12 04:21:14,309 [DEBUG] kashgari - Top-10: ['[PAD]', '[UNK]', '[CLS]', '[SEP]', 'also', 'may', 'used', 'one', 'many', 'first']
Calculating sequence length: 100%|██████████| 331/331 [00:00<00:00, 512671.57it/s]
2020-10-12 04:21:30,982 [DEBUG] kashgari - Calculated sequence length = 142
2020-10-12 04:21:32,523 [DEBUG] kashgari - Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, None)]       0                    

Epoch 1/20
44/44 [==============================] - 39s 884ms/step - loss: 1.0623 - accuracy: 0.6236 - val_loss: 0.6440 - val_accuracy: 0.7500
Epoch 2/20
44/44 [==============================] - 38s 858ms/step - loss: 0.4433 - accuracy: 0.8505 - val_loss: 0.4266 - val_accuracy: 0.8281
Epoch 3/20
44/44 [==============================] - 39s 887ms/step - loss: 0.3824 - accuracy: 0.8654 - val_loss: 0.3168 - val_accuracy: 0.8781
Epoch 4/20
44/44 [==============================] - 52s 1s/step - loss: 0.2520 - accuracy: 0.9116 - val_loss: 0.3127 - val_accuracy: 0.8719
Epoch 5/20
44/44 [==============================] - 56s 1s/step - loss: 0.2383 - accuracy: 0.9165 - val_loss: 0.2914 - val_accuracy: 0.8625
Epoch 6/20
44/44 [==============================] - 59s 1s/step - loss: 0.1786 - accuracy: 0.9386 - val_loss: 0.2883 - val_accuracy: 0.8844
Epoch 7/20
44/44 [==============================] - 36s 828ms/step - loss: 0.1302 - accuracy: 0.9574 - val_loss: 0.3377 - val_accuracy: 0.8719
Epoch 8/

KeyboardInterrupt: ignored

In [35]:
loaded_model = model.load_model('/content/lstm')

2020-10-12 06:40:39,183 [DEBUG] kashgari - ------------------------------------------------
2020-10-12 06:40:39,185 [DEBUG] kashgari - Loaded transformer model's vocab
2020-10-12 06:40:39,186 [DEBUG] kashgari - config_path       : /content/uncased_L-12_H-768_A-12/bert_config.json
2020-10-12 06:40:39,186 [DEBUG] kashgari - vocab_path      : /content/uncased_L-12_H-768_A-12/vocab.txt
2020-10-12 06:40:39,187 [DEBUG] kashgari - checkpoint_path : /content/uncased_L-12_H-768_A-12/bert_model.ckpt
2020-10-12 06:40:39,188 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31

{"class_name": "Functional", "config": {"name": "functional_3", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, null], "dtype": "float32", "sparse": false, "ragged": false, "name": "Input-Token"}, "name": "Input-Token", "inbound_nodes": []}, {"class_name": "InputLayer", "config": {"batch_input_shape": [null, null], "dtype": "float32", "sparse": false, "ragged": false, "name": "Input-Segment"}, "name": "Input-Segment", "inbound_nodes": []}, {"class_name": "Embedding", "config": {"name": "Embedding-Token", "trainable": false, "batch_input_shape": [null, null], "dtype": "float32", "input_dim": 30522, "output_dim": 768, "embeddings_initializer": {"class_name": "TruncatedNormal", "config": {"mean": 0.0, "stddev": 0.02, "seed": null}}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": true, "input_length": null}, "name": "Embedding-Token", "inbound_nodes": [[["Input-Token", 0, 0, {}]]]}, {"class_name": "Em

In [37]:
loaded_model.evaluate(test_x,test_y)

2020-10-12 06:41:05,265 [WARNING] kashgari - Sequence length is None, will use the max length of the samples, which is 128
2020-10-12 06:41:05,298 [DEBUG] kashgari - predict input shape (2, 475, 128) x: 
(array([[  101,  3218,  3262, ...,     0,     0,     0],
       [  101,  3787,  2647, ...,     0,     0,     0],
       [  101,  3570,  2658, ...,     0,     0,     0],
       ...,
       [  101, 29571,  5812, ...,     0,     0,     0],
       [  101,  5069,  9518, ...,     0,     0,     0],
       [  101,  2220, 16215, ...,     0,     0,     0]], dtype=int32), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32))
2020-10-12 06:41:17,993 [DEBUG] kashgari - predict output shape (475, 7)
2020-10-12 06:41:17,996 [DEBUG] kashgari - predict output argmax: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 5 0 0 0 0 0 0 0 0 0 0 0

                  precision    recall  f1-score   support

        Business     0.9608    1.0000    0.9800        49
acadamic_science     0.9799    0.9605    0.9701       152
       community     0.9877    0.9877    0.9877        81
       computing     0.9348    0.9773    0.9556        44
     govermnetal     1.0000    0.9853    0.9926        68
         medical     0.9753    0.9753    0.9753        81

        accuracy                         0.9768       475
       macro avg     0.9731    0.9810    0.9769       475
    weighted avg     0.9772    0.9768    0.9769       475



{'detail': {'Business': {'f1-score': 0.98,
   'precision': 0.9607843137254902,
   'recall': 1.0,
   'support': 49},
  'acadamic_science': {'f1-score': 0.9700996677740865,
   'precision': 0.9798657718120806,
   'recall': 0.9605263157894737,
   'support': 152},
  'accuracy': 0.9768421052631578,
  'community': {'f1-score': 0.9876543209876543,
   'precision': 0.9876543209876543,
   'recall': 0.9876543209876543,
   'support': 81},
  'computing': {'f1-score': 0.9555555555555557,
   'precision': 0.9347826086956522,
   'recall': 0.9772727272727273,
   'support': 44},
  'govermnetal': {'f1-score': 0.9925925925925926,
   'precision': 1.0,
   'recall': 0.9852941176470589,
   'support': 68},
  'macro avg': {'f1-score': 0.976868463147533,
   'precision': 0.973065942866031,
   'recall': 0.9810093539453705,
   'support': 475},
  'medical': {'f1-score': 0.9753086419753086,
   'precision': 0.9753086419753086,
   'recall': 0.9753086419753086,
   'support': 81},
  'weighted avg': {'f1-score': 0.976875558

In [38]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [39]:
achedStopWords = stopwords.words("english")
 
def tokenize(text):
  min_length = 3
  words = map(lambda word: word.lower(), word_tokenize(text))
  words = [word for word in words if word not in achedStopWords]
  tokens = (list(map(lambda token: PorterStemmer().stem(token),
                                   words)))
  p = re.compile('[a-zA-Z]+');
  filtered_tokens =list(filter (lambda token: p.match(token) and
                               len(token) >= min_length,
                               tokens))
  return filtered_tokens

def word_tokenizer():
  tokenize_docs = []
  docs_tokens = docs_pdf_train['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [40]:
def predict(docs):
  tokenize_docs = []
  df=pd.DataFrame([docs],columns=['text'])
  docs_tokens = df['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [71]:
x = predict('A medical encyclopaedia is a comprehensive written compendium that holds information about diseases, medical conditions, tests, symptoms, injuries, and surgeries. It may contain an extensive gallery of medicine-related photographs and illustrations. [1] A medical encyclopaedia provides information to readers about health questions. It may also contain some information about the history of diseases, the development of medical technology uses to detect diseases in its early phase. A licensed physician should be consulted for diagnosis and treatment of any and all medical conditions.')

In [72]:
len(x[0])

49

In [73]:
loaded_model.predict(x)

2020-10-12 06:46:30,903 [DEBUG] kashgari - predict input shape (2, 1, 51) x: 
(array([[  101,   100,   100,   100,  2517,   100,  2907, 12367,   100,
          100,   100,  3231,   100,   100,   100,  2089,  5383,   100,
          100,   100,  9982,   100,   100,   100,   100, 12367,  8068,
         2740,  3160,  2089,  2036,  5383, 12367,   100,   100,  4503,
          100,   100,  2224, 11487,   100,   100,  4403,   100,  7522,
        23363,   100,  3949,   100,   100,   102]], dtype=int32), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]], dtype=int32))
2020-10-12 06:46:30,976 [DEBUG] kashgari - predict output shape (1, 7)
2020-10-12 06:46:30,976 [DEBUG] kashgari - predict output argmax: [2]


['medical']

In [46]:
model.save('lstm_1')

2020-10-12 04:33:06,828 [INFO] kashgari - model saved to /content/lstm_1


'/content/lstm_1'

In [47]:
wandb.save('/content/lstm_1')

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


['wandb/run-20201012_042017-12x51cbi/files/lstm_1']

In [51]:
wandb.save("/content/wandb/latest-run/files/lstm_1/embed_model_weights.h5", base_path="/content/wandb/latest-run/")
wandb.save("/content/wandb/latest-run/files/lstm_1/model_config.json", base_path="/content/wandb/latest-run/")
wandb.save("/content/wandb/latest-run/files/lstm_1/model_weights.h5", base_path="/content/wandb/latest-run/")



['wandb/run-20201012_042017-12x51cbi/files/files/lstm_1/model_weights.h5',
 'wandb/run-20201012_042017-12x51cbi/files/files/lstm_1/model_weights.h5']

In [ ]:
train_df

,Unnamed: 0,title,text
0,0,Academic_Science,"['an', 'academic', 'degree', 'is', 'a', 'quali..."
1,1,Academic_Science,"['an', 'alumnus', 'masculine', 'plural', 'alum..."
2,2,Academic_Science,"['an', 'example', 'of', 'an', 'amateur', 'radi..."
3,3,Academic_Science,"['anthropology', 'is', 'the', 'study', 'of', '..."
4,4,Academic_Science,"['roman', 'ruins', 'lausanne', 'switzerland', ..."
...,...,...,...
154,154,Medical,"['rehabilitation', 'psychology', 'is', 'a', 's..."
155,155,Medical,"['surgeons', 'repairing', 'a', 'ruptured', 'ac..."
156,156,Medical,"['a', 'syndrome', 'is', 'a', 'set', 'of', 'med..."
157,157,Medical,"['therapy', 'often', 'abbreviated', 'tx', 'tx'..."


In [ ]:
import ast
X_train=[ast.literal_eval(i) for i in X_train]
X_test=[ast.literal_eval(i) for i in X_test]

In [ ]:
X_train_filtered_list=[]
X_train_filtered= [] 
for w in X_train:
  for j in w:
    if j not in stop_words: 
      X_train_filtered.append(j)
  X_train_filtered_list.append(X_train_filtered)

In [ ]:
X_train_filtered_list=[]
X_train_filtered= [] 
for w in X_train:
  for j in w:
    if j not in stop_words: 
      X_train_filtered.append(j)
  X_train_filtered_list.append(X_train_filtered)

In [ ]:
X_test_filtered_list=[]
X_test_filtered= [] 
for w in X_test:
  for j in w:
    if j not in stop_words: 
      X_test_filtered.append(j)
  X_test_filtered_list.append(X_test_filtered)

In [ ]:
X_test_filtered_list[0]

In [ ]:
model.fit(X_train_filtered_list,y_train,X_test_filtered_list,y_test,epochs=35,batch_size=20)

In [ ]:
model.save("LSTM")

In [ ]:
!mkdir '/content/LSTM' 
!cp  '/content/drive/My Drive/Model_weights/embed_model_weights.h5' '/content/LSTM'
!cp  '/content/drive/My Drive/Model_weights/model_config.json' '/content/LSTM'
!cp  '/content/drive/My Drive/Model_weights/model_weights.h5' '/content/LSTM'


In [ ]:

loaded_model = BiLSTM_Model.load_model('/content/LSTM/')

2020-09-01 17:51:13,866 [DEBUG] kashgari - ------------------------------------------------
2020-09-01 17:51:13,867 [DEBUG] kashgari - Loaded transformer model's vocab
2020-09-01 17:51:13,868 [DEBUG] kashgari - config_path       : /content/uncased_L-12_H-768_A-12/bert_config.json
2020-09-01 17:51:13,870 [DEBUG] kashgari - vocab_path      : /content/uncased_L-12_H-768_A-12/vocab.txt
2020-09-01 17:51:13,871 [DEBUG] kashgari - checkpoint_path : /content/uncased_L-12_H-768_A-12/bert_model.ckpt
2020-09-01 17:51:13,872 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31

In [ ]:
def predict(docs):
  tokenize_docs = []
  df=pd.DataFrame([docs],columns=['text'])
  docs_tokens = df['text'].tolist()
  length = len(docs_tokens)
  for i in range(length):
    tokenize_docs.append(tokenize(docs_tokens[i]))
  return tokenize_docs

In [ ]:
loaded_model.predict([nltk_tokens])

2020-09-01 18:08:44,835 [DEBUG] kashgari - predict input shape (2, 1, 62) x: 
(array([[  101,   100, 13058,   100,  5658,   100, 14931,  1058, 14931,
         5658,  1058,  5096,   100,  1058,   100,   100,   100,   100,
         1058,   100,  3157,   100,   100, 14931,  1058, 14931,  5658,
          100,  1058,   100,  5096,   100,  1058,   100,   100,   100,
          100,  1058,   100,   100,  2566,  3745, 11138,   100,  1997,
         2048,   100,  3745,  1999,  3988,  2270,  3157,   100,  2765,
         2421,  9313,  5114,  1997,   100,  2030, 14931,   102]],
      dtype=int32), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32))
2020-09-01 18:08:44,907 [DEBUG] kashgari - predict output shape (1, 110)


[['earn']]

In [ ]:
loaded_model.evaluate(toknize_test,test_labels_)

2020-09-01 18:10:17,533 [DEBUG] kashgari - predict input shape (2, 10389, 512) x: 
(array([[ 101, 1045, 2572, ...,    0,    0,    0],
       [ 101, 1045, 2031, ...,    0,    0,    0],
       [ 101, 1045, 2342, ...,    0,    0,    0],
       ...,
       [ 101, 3034, 3013, ...,    0,    0,    0],
       [ 101, 3354, 3919, ...,    0,    0,    0],
       [ 101, 2416, 2730, ...,    0,    0,    0]], dtype=int32), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32))
2020-09-01 18:17:21,525 [DEBUG] kashgari - predict output shape (10389, 110)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support
                earn     0.9709    0.9825    0.9767      1087
                 acq     0.9677    0.9583    0.9630       719
      comp.windows.x     0.4296    0.5964    0.4995       394
    rec.sport.hockey     0.9121    0.7143    0.8011       392
soc.religion.christian     0.5245    0.8010    0.6339       387
     rec.motorcycles     0.9403    0.3214    0.4791       392
comp.sys.ibm.pc.hardware     0.5000    0.0052    0.0102       387
        misc.forsale     0.8287    0.6108    0.7033       388
             sci.med     0.9623    0.5312    0.6846       384
  rec.sport.baseball     0.7622    0.6909    0.7248       385
           sci.crypt     0.4503    0.6518    0.5326       382
           sci.space     0.9820    0.4282    0.5964       383
     sci.electronics     0.5520    0.3603    0.4360       383
comp.os.ms-windows.misc     1.0000    0.0026    0.0052       382
       comp.graphics     0.7683    0.1641    0.2704       384

{'detail': {'acq': {'f1': 0.962962962962963,
   'precision': 0.9676966292134831,
   'recall': 0.9582753824756607,
   'support': 719},
  'alt.atheism': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 313},
  'alum': {'f1': 0.5882352941176471,
   'precision': 0.9090909090909091,
   'recall': 0.43478260869565216,
   'support': 23},
  'barley': {'f1': 0.45454545454545453,
   'precision': 0.625,
   'recall': 0.35714285714285715,
   'support': 14},
  'bop': {'f1': 0.5199999999999999,
   'precision': 0.65,
   'recall': 0.43333333333333335,
   'support': 30},
  'carcass': {'f1': 0.6250000000000001,
   'precision': 0.7142857142857143,
   'recall': 0.5555555555555556,
   'support': 18},
  'castor-oil': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 1},
  'cocoa': {'f1': 0.9714285714285714,
   'precision': 1.0,
   'recall': 0.9444444444444444,
   'support': 18},
  'coconut': {'f1': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 2},
  'coconut-oil': {'f1': 0.0, 'precision': 0.0,

In [ ]:
all_dataframes_test['title'][10360]

['money-fx']

In [ ]:
nltk_tokens = nltk.word_tokenize(text)


In [ ]:
x=all_dataframes_test['text'][10360]


In [ ]:
text=all_dataframes_test['text'][10001]

In [ ]:
text

'jepson corp qtr net shr ct v ct net v sale mln v mln avg shrs mln v mln nine mths shr ct v ct net mln v mln sale mln v mln avg shrs mln v mln qtr per share reflects issuance of two mln share in initial public nine mth result include extraordinary gain of dlrs or ct'

In [ ]:
all_dataframes_train.to_csv("train_allDatasets.csv")
all_dataframes_test.to_csv("test_allDatasets.csv")


In [ ]:
!cp '/content/train_allDatasets.csv' '/content/drive/My Drive/'
!cp '/content/test_allDatasets.csv' '/content/drive/My Drive/'